In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
def financial_statement(year, season, type='綜合損益彙總表'):

    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
        x=1
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
        x=1
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
        x=0
    else:
        print('type does not match')

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season),
    })

    r.encoding = 'utf8'
    dfs = pd.read_html(r.text, header=0)
    #print(len(dfs))
    if x==0 :
        out=dfs[0]
    elif x==1 :
        out=pd.concat(dfs[x:], axis=0, sort=False)
    return out

def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d  #民國轉西元
    
def transform_data(data):
    data[0] = datetime.datetime.strptime(transform_date(data[0]), '%Y/%m/%d')
    data[1] = int(data[1].replace(',', ''))  #把千進位的逗點去除
    data[2] = int(data[2].replace(',', ''))
    data[3] = float(data[3].replace(',', ''))
    data[4] = float(data[4].replace(',', ''))
    data[5] = float(data[5].replace(',', ''))
    data[6] = float(data[6].replace(',', ''))
    data[7] = float(0.0 if data[7].replace(',', '') == 'X0.00' else data[7].replace(',', ''))  # +/-/X表示漲/跌/不比價
    data[8] = int(data[8].replace(',', ''))
    return data

def transform(data):
    return [transform_data(d) for d in data]
def get_stock_history(date, stock_no):
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, stock_no)
    r = requests.get(url)
    data = r.json()
    return transform(data['data'])  #進行資料格式轉換
def get_stock_nowprice(stock_no):    
    now=datetime.datetime.now()
    Y=int(now.strftime('%Y'))
    M=int(now.strftime('%m'))
    formatted_today=now.strftime('%Y%m%d')
    formatted_now=now.strftime("%Y_%m_%d_%H%M%S")
    lst=get_stock_history(formatted_now, stock_no)
    #stock_no='00878'
    now_price=lst[len(lst)-1][6]
    return now_price
def now_Market_Cap():
    dfs=pd.read_html('https://stock.capital.com.tw/z/zm/zmd/zmdb.djhtm?MSCI=0', header=0,flavor='bs4')
    df=dfs[3][dfs[3]['台股權值100大排行榜']!='公司代碼名稱']
    df['公司代號']=df['台股權值100大排行榜'].str[:4]
    return df
def get_company_nowinfo_formgoogle(company_no):
    url = "https://www.google.com/finance/quote/{}:TPE".format(company_no) # PTT Python看板
    response = requests.get(url) # 使用requests的get方法把網頁抓下來
    html_doc = response.text # text屬性就是html文件原始碼
    soup = BeautifulSoup(response.text, "lxml") # 指定lxml作為解析器來建立Beautiful物件
    x=soup.find_all('div', class_='P6K39c', id=None)
    return x
def get_stock_nowprice(company_no):
    x=get_company_nowinfo_formgoogle(company_no)
    pricestr=x[0].get_text()
    pricestr1=pricestr.replace(',','')
    priceint=float(pricestr1[1:])
    return priceint

In [2]:
year=2021
season=1
financial1_Q1=financial_statement(year, season, type='綜合損益彙總表')
financial1_Q1['公司代號']=financial1_Q1['公司代號'].astype(str)
financial2_Q1=financial_statement(year, season, type='資產負債彙總表')
financial2_Q1['公司代號']=financial2_Q1['公司代號'].astype(str)
financial3_Q1=financial_statement(year, season, type='營益分析彙總表')


financial=pd.merge(financial1_Q1,financial3_Q1,how='left')

Market_Cap=now_Market_Cap()
financial1=pd.merge(Market_Cap,financial,how='left')

myfinancial=pd.DataFrame(columns=['公司代號', '公司名稱'])
myfinancial['公司代號']=financial1['公司代號']
myfinancial['公司名稱']=financial1['公司名稱']
myfinancial['營業收入']=financial1['營業收入']
myfinancial['營業收入(百萬元)']=financial1['營業收入(百萬元)']
myfinancial['基本每股盈餘（元）']=financial1['基本每股盈餘（元）']
myfinancial['市值']=financial1['台股權值100大排行榜.1']
myfinancial



<ipython-input-1-b7f124948b44>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['公司代號']=df['台股權值100大排行榜'].str[:4]


,公司代號,公司名稱,營業收入,營業收入(百萬元),基本每股盈餘（元）,市值
0,2330,台積電,3.624102e+08,362410.23,5.39,16491722
1,2454,聯發科,1.080330e+08,108032.98,16.21,1718754
2,2317,鴻海,1.347113e+09,1347112.82,2.03,1414025
3,6505,台塑化,1.302997e+08,130299.67,1.88,915445
4,2412,中華電,5.010100e+07,50101.00,1.14,911500
...,...,...,...,...,...,...
95,2633,台灣高鐵,9.867362e+06,9867.36,0.24,74775
96,2227,裕日車,7.900885e+06,7900.89,2.70,73650
97,2492,華新科,1.050591e+07,10505.91,4.06,73114
98,2915,潤泰全,6.424860e+05,642.49,12.80,69466


In [3]:
import time
company_no=myfinancial['公司代號'].tolist()

stock_price=[]
for i in range(len(company_no)):
    print(company_no[i])
    now_price=get_stock_nowprice(int(company_no[i]))
    stock_price.append(now_price)
stock_price

2330
2454
2317
6505
2412
2881
2882
2303
2308
1303
1301
2603
2891
2002
2886
1326
3711
2884
1216
5871
2615
5880
2382
3045
2609
6415
2207
2892
2883
2885
8046
3037
2395
2880
1101
3034
2912
3008
2379
2357
2327
2887
2408
4904
5876
8454
9910
2409
1590
2890
2801
4938
3481
6669
1476
1102
2888
1402
2301
2345
2610
9945
2377
6781
2618
6409
2344
9921
2474
8464
2347
2105
2324
3532
5269
9904
3702
3231
3189
2356
2049
2360
4958
2027
1605
2383
2376
9914
2353
1229
2354
2834
2337
6239
3533
2633
2227
2492
2915
3036


[641.0,
 1085.0,
 103.0,
 95.6,
 117.0,
 75.5,
 63.0,
 61.5,
 269.0,
 85.5,
 104.5,
 110.0,
 27.25,
 33.45,
 36.9,
 78.6,
 100.0,
 28.85,
 67.6,
 244.0,
 145.5,
 26.2,
 93.0,
 100.5,
 95.1,
 3565.0,
 598.0,
 25.1,
 18.2,
 25.15,
 477.0,
 200.5,
 382.0,
 21.7,
 46.8,
 473.0,
 265.5,
 2060.0,
 531.0,
 360.0,
 451.0,
 19.7,
 72.0,
 65.0,
 47.15,
 1165.0,
 221.5,
 19.6,
 914.0,
 16.4,
 17.6,
 69.9,
 16.65,
 985.0,
 599.0,
 44.15,
 10.95,
 28.6,
 63.8,
 274.5,
 22.8,
 63.4,
 154.0,
 1425.0,
 23.55,
 1420.0,
 29.6,
 312.0,
 155.5,
 375.5,
 68.0,
 34.55,
 25.25,
 276.5,
 1625.0,
 31.95,
 54.5,
 31.6,
 198.0,
 25.45,
 261.5,
 206.0,
 95.3,
 44.2,
 25.25,
 265.0,
 142.0,
 290.0,
 28.5,
 61.2,
 60.4,
 10.25,
 38.6,
 98.1,
 712.0,
 28.15,
 245.0,
 151.0,
 93.8,
 86.5]

In [4]:
myfinancial['當前股價']=stock_price

In [6]:
myfinancial['本益比']=myfinancial['當前股價']/myfinancial['基本每股盈餘（元）']
myfinancial.to_excel(r'myfinancial.xlsx')

In [15]:
lst=get_stock_history('20220101', '2330')
now_price=lst[len(lst)-1][6]
now_price

ConnectionError: HTTPConnectionPool(host='www.twse.com.tw', port=80): Max retries exceeded with url: /exchangeReport/STOCK_DAY?date=20220101&stockNo=2330 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B0F6ADA940>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))

In [7]:
financial_Q1[0]

,0,1,2,3,4,5,6
0,公司代號,公司名稱,營業收入(百萬元),毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
1,1101,台泥,22037.65,25.07,19.09,22.03,16.93
2,1102,亞泥,17877.39,24.39,20.83,26.54,21.53
3,1103,嘉泥,544.53,-1.88,-24.84,-20.35,-22.09
4,1104,環泥,1440.50,17.41,9.83,17.96,16.46
...,...,...,...,...,...,...,...
984,9944,新麗,706.95,28.47,8.98,8.90,6.70
985,9945,潤泰新,5237.44,19.54,10.96,134.08,131.34
986,9946,三發地產,146.05,25.99,6.63,8.81,6.37
987,9955,佳龍,459.52,2.27,-3.09,-4.82,-4.82


In [21]:
company_no=all1time['公司代號'].tolist()
company_name=all1time['公司名稱'].tolist()
for i in range(len(company_no)):
    

2810

In [14]:
df.columns.values.tolist()

['公司代號',
 '公司名稱',
 '利息淨收益',
 '利息以外淨損益',
 '呆帳費用、承諾及保證責任準備提存',
 '營業費用',
 '繼續營業單位稅前淨利（淨損）',
 '所得稅費用（利益）',
 '繼續營業單位本期稅後淨利（淨損）',
 '停業單位損益',
 '合併前非屬共同控制股權損益',
 '本期稅後淨利（淨損）',
 '其他綜合損益（稅後）',
 '合併前非屬共同控制股權綜合損益淨額',
 '本期綜合損益總額（稅後）',
 '淨利（損）歸屬於母公司業主',
 '淨利（損）歸屬於共同控制下前手權益',
 '淨利（損）歸屬於非控制權益',
 '綜合損益總額歸屬於母公司業主',
 '綜合損益總額歸屬於共同控制下前手權益',
 '綜合損益總額歸屬於非控制權益',
 '基本每股盈餘（元）',
 '收益',
 '支出及費用',
 '營業利益',
 '營業外損益',
 '稅前淨利（淨損）',
 '繼續營業單位本期淨利（淨損）',
 '本期淨利（淨損）',
 '本期其他綜合損益（稅後淨額）',
 '本期綜合損益總額',
 '淨利（淨損）歸屬於共同控制下前手權益',
 '營業收入',
 '營業成本',
 '原始認列生物資產及農產品之利益（損失）',
 '生物資產當期公允價值減出售成本之變動利益（損失）',
 '營業毛利（毛損）',
 '未實現銷貨（損）益',
 '已實現銷貨（損）益',
 '營業毛利（毛損）淨額',
 '其他收益及費損淨額',
 '營業利益（損失）',
 '營業外收入及支出',
 '其他綜合損益（淨額）',
 '淨利（淨損）歸屬於母公司業主',
 '淨利（淨損）歸屬於非控制權益',
 '利息以外淨收益',
 '淨收益',
 '保險負債準備淨變動',
 '繼續營業單位稅前損益',
 '所得稅（費用）利益',
 '繼續營業單位稅前純益（純損）',
 '繼續營業單位本期純益（純損）',
 '其他綜合損益（稅後淨額）',
 '收入',
 '支出',
 '其他綜合損益']

In [9]:
import pandas as pd
dfs=pd.read_html('https://stock.capital.com.tw/z/zm/zmd/zmdb.djhtm?MSCI=0', header=0,flavor='bs4')
dfs[3]

,台股權值100大排行榜,台股權值100大排行榜.1,台股權值100大排行榜.2,台股權值100大排行榜.3,台股權值100大排行榜.4,台股權值100大排行榜.5,台股權值100大排行榜.6
0,公司代碼名稱,市值(百萬元),佔台股大盤比重(%),累加比重(%),大盤β值,電子β值,金融β值
1,2330台積電,16491722,30.50%,30.50%,1.07,0.97,NaN
2,2454聯發科,1718754,3.18%,33.68%,1.39,1.09,NaN
3,2317鴻海,1414025,2.61%,36.29%,1.19,1.20,NaN
4,6505台塑化,915445,1.69%,37.98%,0.77,NaN,NaN
...,...,...,...,...,...,...,...
96,2633台灣高鐵,74775,0.14%,80.86%,0.57,NaN,NaN
97,2227裕日車,73650,0.14%,81.00%,0.40,NaN,NaN
98,2492華新科,73114,0.14%,81.14%,1.69,1.23,NaN
99,2915潤泰全,69466,0.13%,81.27%,0.87,NaN,NaN


In [11]:
df=dfs[3][dfs[3]['台股權值100大排行榜']!='公司代碼名稱']

In [12]:
df['公司代號']=df['台股權值100大排行榜'].str[:4]

<ipython-input-12-6ed391c614e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['公司代號']=df['台股權值100大排行榜'].str[:4]


In [13]:
df

,台股權值100大排行榜,台股權值100大排行榜.1,台股權值100大排行榜.2,台股權值100大排行榜.3,台股權值100大排行榜.4,台股權值100大排行榜.5,台股權值100大排行榜.6,公司代號
1,2330台積電,16491722,30.50%,30.50%,1.07,0.97,NaN,2330
2,2454聯發科,1718754,3.18%,33.68%,1.39,1.09,NaN,2454
3,2317鴻海,1414025,2.61%,36.29%,1.19,1.20,NaN,2317
4,6505台塑化,915445,1.69%,37.98%,0.77,NaN,NaN,6505
5,2412中華電,911500,1.69%,39.67%,0.17,0.62,NaN,2412
...,...,...,...,...,...,...,...,...
96,2633台灣高鐵,74775,0.14%,80.86%,0.57,NaN,NaN,2633
97,2227裕日車,73650,0.14%,81.00%,0.40,NaN,NaN,2227
98,2492華新科,73114,0.14%,81.14%,1.69,1.23,NaN,2492
99,2915潤泰全,69466,0.13%,81.27%,0.87,NaN,NaN,2915
